In [39]:
import pandas as pd
import numpy as np
from Levenshtein import distance


In [40]:
import os
input_dir = './cleaned_ab_dfs/'
os.listdir(input_dir)

['absci_denovo-HER2_binders_cleaned_24-03-05.csv',
 'andrews_neut_H1N1_Cal09_v2_24-03-07.csv',
 'bornholdt_binders_v2_24-03-07.csv',
 'covabdab_input_seqs_v2_24-03-12.csv',
 'elaine_ebola_binders_v2_24-03-10.csv',
 'HCV_weber_seqs_v2_24-03-07.csv',
 'joyce_h5n1_vaccine_24-01-04.csv',
 'libra-seq_10618_binders_v2_24-03-10.csv',
 'published_LIBRA-seq_binders_genV2_24-03-09.csv',
 'rsv_Gilman_rsvA-B_seqs_v2_24-03-07.csv',
 'sabdab_detagged_24-03-12.csv',
 'zurbuchen_binders_v2_24-03-08.csv']

In [41]:
def lev_identity(seq1, seq2):
    lev_dist = distance(seq1, seq2)

    max_len = max(len(seq1), len(seq2))

    norm_dist = lev_dist / max_len

    norm_identity = 1 - norm_dist

    return norm_identity

In [42]:
# concat_data = pd.concat([pd.read_csv(input_dir + file) for file in os.listdir(input_dir)])

In [43]:
# Processed
zurbuchen = pd.read_csv(input_dir + 'zurbuchen_binders_v2_24-03-08.csv', index_col=0)
zurbuchen['source'] = 'Zurbuchen'


bornholdt = pd.read_csv(input_dir + 'bornholdt_binders_v2_24-03-07.csv', index_col=0)
bornholdt['antigen_name'] = 'EBOV' + bornholdt['Antigen']
bornholdt['source'] = 'Bornholdt'

covabdab = pd.read_csv(input_dir + 'covabdab_input_seqs_v2_24-03-12.csv', index_col = 0)
covabdab = covabdab[~covabdab['antigen_seq'].isna()] # Never mapped in MERS, removing here
covabdab['source'] = 'CoVAbDab'
covabdab['antigen_name'] = covabdab['antigen_name'] +  ' ' + covabdab['Epitope']

absci_denovo = pd.read_csv(input_dir + 'absci_denovo-HER2_binders_cleaned_24-03-05.csv', index_col = 0)

gilman = pd.read_csv(input_dir + 'rsv_Gilman_rsvA-B_seqs_v2_24-03-07.csv', index_col=0)
gilman['antigen_name'] = 'RSV'
gilman['source'] = 'Gilman'

andrews = pd.read_csv(input_dir + 'andrews_neut_H1N1_Cal09_v2_24-03-07.csv', index_col=0)
andrews['antigen_name'] = 'H1N1'
andrews['source'] = 'Andrews'

# Need more work

# need to map AA seqs
pub_lseq = pd.read_csv(input_dir + 'published_LIBRA-seq_binders_genV2_24-03-09.csv', index_col = 0)
pub_lseq['source'] = 'Pub. LIBRA-seq'
pub_lseq['source'] = pub_lseq['Vantage_ID']
pub_lseq['antigen_name'] = pub_lseq['Antigen']

aa10618 = pd.read_csv(input_dir + 'libra-seq_10618_binders_v2_24-03-10.csv', index_col = 0)
aa10618['source'] = '10618-AA LIBRA-seq'
aa10618['antigen_name'] = aa10618['Antigen']

weber = pd.read_csv(input_dir + 'HCV_weber_seqs_v2_24-03-07.csv', index_col=0)
weber['antigen_name'] = 'HCV'
weber['source'] = 'Weber'

elaine = pd.read_csv(input_dir + 'elaine_ebola_binders_v2_24-03-10.csv', index_col=0)
elaine['source'] = 'Elaine / Crowe'


# Untouched


joyce = pd.read_csv(input_dir + 'joyce_h5n1_vaccine_24-01-04.csv', index_col=0)
joyce['antigen_name'] = 'H5N1'
joyce['source'] = 'Joyce'



# ehrhardt: IDs don't match between GenBank and supplement. Could potentially map using CDRH3, but tables with CDRH3s are in PDF form and not easy to extract.
# Liu: couldn't find ELISA data. Might could be curated later


In [44]:
pub_lseq['source'].value_counts()

5404       430
5885       383
150517     347
5812       207
4647-2      67
4647-1      44
7128-2      42
4591-1      30
6420        27
5317        17
1655331     11
6421-1       5
5318-1       3
5318-2       3
Name: source, dtype: int64

In [45]:
# Need to remove heldout seqs
sabdab = pd.read_csv(input_dir + 'sabdab_detagged_24-03-12.csv')
sabdab['pdb_id'] = sabdab['PDB-ID_HC_LC_AG'].apply(lambda x: x.split('_')[0].lower())
sabdab.index = sabdab['pdb_id']
sabdab_meta = pd.read_csv('Clean_20231219_SAbDab_SingleChainAG_I.csv', index_col=0)
# sabdab = sabdab[~sabdab['AG_AA'].apply(lambda x: '|' in x)] #### NOTE: NEED TO FIX THISS!

sabdab['antigen_seq'] = sabdab['detagged_ant_seq']
sabdab['antigen_name'] = sabdab_meta['antigen_name']
sabdab['source'] = 'SAbDab'

In [46]:
ang2_alexis = 'MPMGSLQPLATLYLLGMLVASVLAAGSEQISFRDCAEVFKSGHTTNGIYTLTFPNSTEEIKAYCDMEAGGGGWTIIQRREDGSVDFQRTWKEYKVGFGNPSGEYWLGNEFVSQLTNQQRYVLKIHLKDWEGNEAYSLYEHFYLSSEELNYRIHLKGLTGTAGKISSISQPGNDFSTKDGDNDKCICKCSQMLTGGWWFDACGPSNLNGMYYPQRQNTNKFNGIKWYYWKGSGYSLKATTMMIRPADFGNSGG'
vegf_monomer_alexis = 'MPMGSLQPLATLYLLGMLVASVLAGQNHHEVVKFMDVYQRSYCHPIETLVDIFQEYPDEIEYIFKPSCVPLMRCGGCCNDEGLECVPTEESNITMQIMRIKPHQGQHIGEMSFLQHNKCECRPKKDGG'
vegf_dimer = 'MPMGSLQPLATLYLLGMLVASVLAMPMAEGGGQNHHEVVKFMDVYQRSYCHPIETLVDIFQEYPDEIEYIFKPSCVPLMRCGGCCNDEGLECVPTEESNITMQIMRIKPHQGQHIGEMSFLQHNKCECRPKKDRARQEKCDKPRRGSTSGSGKSSEGKGMPMAEGGGQNHHEVVKFMDVYQRSYCHPIETLVDIFQEYPDEIEYIFKPSCVPLMRCGGCCNDEGLECVPTEESNITMQIMRIKPHQGQHIGEMSFLQHNKCECRPKKDRARQEKCDKPRRGG'

### SAbDab
* Dropping VEGF (except isoform) and Ang2

In [47]:
sabdab['ang2_ID'] = sabdab['antigen_noSignalP'].apply(lambda x: lev_identity(x, ang2_alexis))

ang2_seqs = sabdab[sabdab['ang2_ID'] > 0.3]
print(len(ang2_seqs))
print(ang2_seqs['antigen_name'])
sabdab = sabdab[sabdab['ang2_ID'] < 0.3]

1
pdb_id
4zfg    angiopoietin-2
Name: antigen_name, dtype: object


In [48]:
sabdab[sabdab['ang2_ID'] > 0.3]

,PDB-ID_HC_LC_AG,PDB-ID_HC_LC_AG.1,HV,HJ,CDRH3,HC_AA,HC_QUALITY_CHECK,LV,LJ,CDRL3,LC_AA,LC_QUALITY_CHECK,AG_AA,antigen_noSignalP,pdb_id,detagged_ant_seq,antigen_seq,antigen_name,source,ang2_ID
pdb_id,,,,,,,,,,,,,,,,,,,,


In [50]:
sabdab['VEGF_monomer_ID'] = sabdab['antigen_noSignalP'].apply(lambda x: lev_identity(x, vegf_monomer_alexis))

vegf_m_seqs = sabdab[sabdab['VEGF_monomer_ID'] > 0.3]
print(len(vegf_m_seqs))

# This is VEGF isoform we left in before
vegf_m_seqs = vegf_m_seqs[vegf_m_seqs['antigen_name'] != 'unknown']

sabdab.drop(vegf_m_seqs.index, inplace=True)
print(sabdab[sabdab['VEGF_monomer_ID'] > 0.3]['antigen_name'])
(vegf_m_seqs['antigen_name'].unique())

10
pdb_id
6t9d    unknown
Name: antigen_name, dtype: object


array(['vascular endothelial growth factor a',
       'isoform l-vegf206 of vascular endothelial growth factor a'],
      dtype=object)

In [51]:
sabdab[sabdab['VEGF_monomer_ID'] > 0.3]

,PDB-ID_HC_LC_AG,PDB-ID_HC_LC_AG.1,HV,HJ,CDRH3,HC_AA,HC_QUALITY_CHECK,LV,LJ,CDRL3,...,LC_QUALITY_CHECK,AG_AA,antigen_noSignalP,pdb_id,detagged_ant_seq,antigen_seq,antigen_name,source,ang2_ID,VEGF_monomer_ID
pdb_id,,,,,,,,,,,,,,,,,,,,,
6t9d,6T9D_A_B_D,6T9D_A_B_D,IGHV3-23,IGHJ5,ARDDGYFDT,DLQLVESGGGLVKPGGSLRLSCAADGWWFGYTDMSWVRQAPGKGLE...,True,IGKV1-NL1,IGKJ2,QQYRYHPYT,...,True,APMAEGGGQNHHEVVKFMDVYQRSYCHPIETLVDIFQEYPDEIEYI...,APMAEGGGQNHHEVVKFMDVYQRSYCHPIETLVDIFQEYPDEIEYI...,6t9d,APMAEGGGQNHHEVVKFMDVYQRSYCHPIETLVDIFQEYPDEIEYI...,APMAEGGGQNHHEVVKFMDVYQRSYCHPIETLVDIFQEYPDEIEYI...,unknown,SAbDab,0.194444,0.75


In [52]:
sabdab['VEGF_dimer_ID'] = sabdab['antigen_noSignalP'].apply(lambda x: lev_identity(x, vegf_dimer))

vegf_d_seqs = sabdab[sabdab['VEGF_dimer_ID'] > 0.5]
print(len(vegf_d_seqs))
(vegf_d_seqs['antigen_name'].unique())

0


array([], dtype=object)

In [14]:
# sabdab_meta = pd.read_csv('./SAbDab/sabdab_summary_all.tsv', sep='\t', index_col=0)
# sabdab_meta = sabdab_meta[sabdab_meta['heavy_species'] == 'homo sapiens']
# sabdab_meta = sabdab_meta[sabdab_meta['light_species'] == 'homo sapiens']
# sabdab_meta['light_ctype'].value_counts()
# sabdab_meta.shape

In [53]:
sabdab = sabdab.rename(columns={'HC_AA':'VH_AA', 'LC_AA':'VL_AA'})

In [ ]:
# viral_spikes = [i for i in sabdab['antigen_name'].dropna().unique() if ('spike' in i) or ('spike' in i)]
# envelope_proteins = [i for i in sabdab['antigen_name'].dropna().unique() if 'nv' in i]
# peptides = [i for i in sabdab['antigen_name'].dropna().unique() if 'peptide' in i]
# glycoproteins = [i for i in sabdab['antigen_name'].dropna().unique() if 'gp' in i]

### Concatenating data

In [54]:
concat_data = pd.concat([andrews, bornholdt, weber, joyce, gilman, sabdab, absci_denovo, pub_lseq, aa10618, elaine,zurbuchen, covabdab]) #
concat_data.shape

(18852, 55)

In [55]:
concat_data[concat_data['VH_AA'].isna()]['source'].value_counts()

Series([], Name: source, dtype: int64)

In [59]:
# concat_data[concat_data['antigen_seq'].isna()]['antigen_name'].value_counts()

In [56]:
concat_data = concat_data[~concat_data['antigen_seq'].isna()]

In [60]:
concat_data = concat_data[['VH_AA', 'VL_AA', 'antigen_seq', 'source', 'antigen_name']]

In [64]:
concat_data.to_csv('concat_inputs_preTag_v2_24-03-12.csv')

In [65]:
# concat_data.shape

(18648, 5)

In [58]:
# concat_data.groupby('source')['antigen_name'].value_counts().to_clipboard()

In [22]:
# concat_data[concat_data['antigen_seq'].isna()]['antigen_name'].value_counts()

Series([], Name: antigen_name, dtype: int64)

In [67]:
concat_data['source'].value_counts()

CoVAbDab              10043
SAbDab                 2248
10618-AA LIBRA-seq     1924
Zurbuchen              1011
Gilman                  479
Joyce                   454
5404                    410
5885                    383
Bornholdt               293
Absci_denovo            258
150517                  245
Weber                   218
5812                    207
Elaine / Crowe          170
Andrews                 138
4647-2                   47
4647-1                   30
4591-1                   29
6420                     27
5317                     17
1655331                  11
6421-1                    5
5318-1                    1
Name: source, dtype: int64